In [1]:
import scan_csv
from LabelGenerator import *
from client import OIDClient
from FastSlidingWindow import *
from Util import *

ld = scan_csv.open_dicts()
train_image_codec = ld['train codec']


Loading dicts...
Total time in fractional seconds: 22.875


In [2]:
print(ld.keys())

dict_keys(['class names', 'class encodings', 'train codec', 'train class', 'val codec', 'val class'])


In [3]:
car_images = ld['train class'][ld['class encodings']['Car']][1000:1001]
print(car_images[0])
print(len(car_images))

95535a40d2762f71
1


In [4]:
import tensorflow as tf
import numpy as np
tf.enable_eager_execution()
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
tf.Session(config=config).close()

In [5]:
def calc_iou(Y_pred, Y):
    zeros = np.zeros([Y_pred.shape[0], Y_pred.shape[1], Y_pred.shape[2], 1], dtype=np.float32)
    y2 = tf.reshape(Y_pred[:, :, :, 2], [Y_pred.shape[0], Y_pred.shape[1], Y_pred.shape[2], 1])
    y4 = tf.reshape(Y_pred[:, :, :, 4], [Y_pred.shape[0], Y_pred.shape[1], Y_pred.shape[2], 1])
    pred_x_w = tf.where(y2 > 0.0, y2, zeros) 
    pred_x_h = tf.where(y4 > 0.0, y4, zeros) 
    
    x1_t = Y_pred[:, :, :, 1:2] - Y[:, :, :, 2:3] / 2.0
    x2_t = Y_pred[:, :, :, 1:2] + Y[:, :, :, 2:3] / 2.0
    
    y1_t = Y_pred[:, :, :, 3:4] - Y[:, :, :, 4:5] / 2.0
    y2_t = Y_pred[:, :, :, 3:4] + Y[:, :, :, 4:5] / 2.0
    
    x1_p = Y_pred[:, :, :, 1:2] - pred_x_w / 2.0
    x2_p = Y_pred[:, :, :, 1:2] + pred_x_w / 2.0
    
    y1_p = Y_pred[:, :, :, 3:4] - pred_x_h / 2.0
    y2_p = Y_pred[:, :, :, 3:4] + pred_x_h / 2.0
    
    cond1 = x2_t < x1_p
    cond2 = x2_p < x1_t
    cond3 = y2_t < y1_p
    cond4 = y2_p < y1_t
    cond_all = tf.logical_or(tf.logical_or(tf.logical_or(cond1, cond2), cond3), cond4)
    
    ious_np = np.zeros([Y_pred.shape[0], Y_pred.shape[1], Y_pred.shape[2], 1], dtype=np.float32)
    
    '''
    far_x = np.min([x2_t, x2_p])
    near_x = np.max([x1_t, x1_p])
    far_y = np.min([y2_t, y2_p])
    near_y = np.max([y1_t, y1_p])
    '''
    
    far_x = tf.where(x2_t < x2_p, x2_t, x2_p)
    near_x = tf.where(x1_t > x1_p, x1_t, x1_p)
    far_y = tf.where(y2_t < y2_p, y2_t, y2_p)
    near_y = tf.where(y1_t > y1_p, y1_t, y1_p)
    
    inter_area = (far_x - near_x + 1.0) * (far_y - near_y + 1.0)
    true_box_area = (x2_t - x1_t + 1.0) * (y2_t - y1_t + 1.0)
    pred_box_area = (x2_p - x1_p + 1.0) * (y2_p - y1_p + 1.0)
    iou = inter_area / (true_box_area + pred_box_area - inter_area)
    iou = tf.where(cond_all, ious_np, iou)
    return iou
    


In [6]:
from threading import Thread, Lock

class Gatherer(threading.Thread):
    def __init__(self, instances, ld, client, stride, size):
        self.instances = instances
        self.ld = ld
        self.client = client
        self.stride = stride
        self.size = size
        threading.Thread.__init__(self)
        self.lock = Lock()
        self.items = []
    
    def run(self):
        index = 0
        while True:
            instance = self.instances[index]
            
#asdef gather(instances, ld, client, stride, size):
#    index = 0
#    while True:
#        instance = instances[index]
#        index += 1
#        if index >= len(instances):
#            index = 0
#        X, Y, image, cmap = label_image(instance, ld['train codec'], ld['class names'], client, stride, size)
#        yield X, Y, image, cmap

In [7]:
w_val = 0.01
W_M_1 = tf.Variable(np.random.uniform(-w_val, w_val, [4, 4, 3, 32]), dtype=tf.float32, name='WM1')
W_M_1_1 = tf.Variable(np.random.uniform(-w_val, w_val, [4, 4, 32, 64]), dtype=tf.float32, name='WM1')
W_M_2 = tf.Variable(np.random.uniform(-w_val, w_val, [3, 3, 64, 128]), dtype=tf.float32, name='WM2')
W_M_2_2 = tf.Variable(np.random.uniform(-w_val, w_val, [3, 3, 128, 64]), dtype=tf.float32, name='WM2')
W_M_3 = tf.Variable(np.random.uniform(-w_val, w_val, [7*7*64, 500]), dtype=tf.float32, name='WM3')
W_M_4_F = tf.Variable(np.random.uniform(-w_val, w_val, [500, 1*5]), dtype=tf.float32, name='WM4F')

In [10]:
client = OIDClient('192.168.1.31', 33333)

loss_value = tf.Variable(0.0, dtype=tf.float32)

optimizer = tf.train.AdamOptimizer(learning_rate=0.001)

def mean_squared(inputs, targets):
  error = inputs - targets
  return tf.reduce_mean(tf.square(error))

async def train():
    async for items in gather(car_images, ld, client, 30, 50):
        X, Y, image, cmap = items[0], items[1], items[2], items[3]
    
        loss = 0.0
        #X, Y, image, cmap = label_image(instance, train_image_codec, ld['class names'], client, 30, 50)
        #X, Y, image, cmap = gather(car_images, ld, client, 30, 50)

        X = np.array(X, dtype=np.float32)
        if np.max([image.shape[0], image.shape[1]]) >= 2000:
            image = None

        if image is not None:
            # ONLY CAR
            Y = Y[:, :, 570:571, :]

            iou_sum = 0.0
            iou_mean = 0.0

            with tf.GradientTape() as tape:
                # ENCODE
                X_flat = tf.reshape(X, [X.shape[0]*X.shape[1], X.shape[2], X.shape[3], X.shape[4]])
                encode_conv = tf.nn.leaky_relu(tf.nn.conv2d(X_flat, W_M_1, [1, 2, 2, 1], 'SAME'))
                encode_conv = tf.nn.leaky_relu(tf.nn.conv2d(encode_conv, W_M_1_1, [1, 2, 2, 1], 'SAME'))
                share_conv = encode_conv

                share_conv = tf.nn.leaky_relu(tf.nn.conv2d(share_conv, W_M_2, [1, 1, 1, 1], 'SAME'))
                share_conv = tf.nn.leaky_relu(tf.nn.conv2d(share_conv, W_M_2_2, [1, 2, 2, 1], 'SAME'))
                share_flat = tf.reshape(share_conv, [share_conv.shape[0], W_M_3.shape[0]])

                flat_1 = tf.nn.leaky_relu(tf.matmul(share_flat, W_M_3))
                Y_pred = tf.matmul(flat_1, W_M_4_F)
                class_pred = tf.nn.sigmoid(Y_pred[:, 0])
                Y_pred = tf.stack([class_pred, Y_pred[:, 1], Y_pred[:, 2], Y_pred[:, 3], 
                                   Y_pred[:, 4]], axis=1)
                Y_pred = tf.reshape(Y_pred, Y.shape)


                iou = calc_iou(Y_pred, Y)
                iou_1 = 1.0 - tf.reduce_mean(iou)

                ms = mean_squared(Y_pred, Y)
                loss_value = ms + iou_1

            loss += loss_value.numpy()
            params = [W_M_1, W_M_1_1, W_M_2, W_M_2_2, W_M_3, W_M_4_F]
            optimizer.apply_gradients(zip(tape.gradient(loss_value, params), params),
                                global_step=tf.train.get_or_create_global_step())

            img_orig = draw_from_label(image, Y, cmap, 50, 0.0, True)
            img_pred = draw_from_label(image, Y_pred.numpy(), cmap, 50, 0.1, True)

            #print(Y_pred.numpy()[5, 5])
            cv2.imshow('orig', img_orig)
            cv2.imshow('pred', img_pred)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                cv2.destroyAllWindows()
                break
        print(loss)

asyncio.ensure_future(train())

Started client thread


<Task pending coro=<train() running at <ipython-input-10-031395a39136>:11>>

2.4457430839538574
2.554454803466797
2.472532272338867
2.4946417808532715
2.484710216522217
2.471907138824463
2.4811484813690186
2.4871020317077637
2.4754106998443604
2.4670186042785645
2.4669997692108154
2.4646170139312744
2.457185983657837
2.452284812927246
2.4610137939453125
2.448901891708374
2.4528019428253174
2.455519914627075
2.451327085494995
2.4518961906433105
2.4526867866516113
2.449807643890381
2.4456067085266113
2.4472463130950928
2.446733236312866
2.4436535835266113
2.4442191123962402
2.444634437561035
2.443697214126587
2.442535400390625
2.4425344467163086
2.4394595623016357
2.44111967086792
2.4388680458068848
2.445573568344116
2.4406163692474365
2.4433560371398926
2.444767713546753
2.441748857498169
2.439906597137451
2.444638252258301
2.4394335746765137
2.441296339035034
2.442763090133667
2.443516731262207
2.44334077835083
2.4429023265838623
2.4414725303649902
2.4398224353790283
2.4387686252593994
2.4391074180603027
2.4386658668518066
2.4394705295562744
2.4394335746765137
